#DDL

In [0]:
%sql
create catalog if not exists sales_catalog;

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS sales_catalog.sales_bronze")
spark.sql("CREATE DATABASE IF NOT EXISTS sales_catalog.sales_silver")
spark.sql("CREATE DATABASE IF NOT EXISTS sales_catalog.sales_gold")

#bronze layer setup

In [0]:
data = [
    (1, "John Doe", "New York", "NY"),
    (2, "Jane Smith", "Los Angeles", "CA"),
    (3, "Michael Johnson", "Chicago", "IL"),
    (4, "Emily Davis", "Houston", "TX"),
    (5, "David Wilson", "Phoenix", "AZ")
]
columns = ["customer_id", "name", "city", "state"]
df = spark.createDataFrame(data, columns)

df.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_bronze.customers")

In [0]:
product_data = [
    (1, "Laptop", "Electronics", 1200.00),
    (2, "Smartphone", "Electronics", 800.00),
    (3, "Tablet", "Electronics", 400.00),
    (4, "Headphones", "Accessories", 150.00),
    (5, "Keyboard", "Accessories", 50.00),
    (6, "Mouse", "Accessories", 25.00),
    (7, "Desk Chair", "Furniture", 200.00),
    (8, "Monitor", "Electronics", 300.00),
    (9, "Printer", "Electronics", 150.00),
    (10, "Smartwatch", "Wearables", 250.00)
]
product_columns = ["product_id", "product_name", "category", "unit_price"]
product_df = spark.createDataFrame(product_data, product_columns)

product_df.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_bronze.products")


In [0]:
from pyspark.sql.functions import to_date

order_data = [
    (1, 1, "2025-01-03"),
    (2, 2, "2025-01-10"),
    (3, 3, "2025-01-17"),
    (4, 4, "2025-01-24"),
    (5, 5, "2025-01-31"),
    (6, 1, "2025-02-05"),
    (7, 2, "2025-02-10"),
    (8, 3, "2025-02-22"),
    (9, 4, "2025-02-24"),
    (10, 5, "2025-02-28")
]
order_columns = ["order_id", "customer_id", "order_date"]
order_df = spark.createDataFrame(order_data, order_columns)\
                 .withColumn("order_date", to_date("order_date", "yyyy-MM-dd"))

order_df.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_bronze.orders")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType

order_item_data = [
    (1,1,7),(1,2,2),(1,3,9),(1,4,5),(1,5,1),(1,6,6),(1,7,10),(1,8,3),(1,9,4),(1,10,8),
    (2,1,3),(2,2,6),(2,3,1),(2,4,9),(2,5,4),(2,6,2),(2,7,8),(2,8,5),(2,9,10),(2,10,7),
    (3,1,2),(3,2,10),(3,3,7),(3,4,1),(3,5,6),(3,6,3),(3,7,5),(3,8,4),(3,9,9),(3,10,8),
    (4,1,8),(4,2,4),(4,3,6),(4,4,1),(4,5,10),(4,6,5),(4,7,2),(4,8,3),(4,9,7),(4,10,9),
    (5,1,9),(5,2,2),(5,3,4),(5,4,8),(5,5,6),(5,6,1),(5,7,10),(5,8,5),(5,9,3),(5,10,7),
    (6,1,6),(6,2,3),(6,3,7),(6,4,5),(6,5,2),(6,6,10),(6,7,1),(6,8,9),(6,9,8),(6,10,4),
    (7,1,1),(7,2,7),(7,3,5),(7,4,3),(7,5,9),(7,6,6),(7,7,2),(7,8,10),(7,9,8),(7,10,4),
    (8,1,5),(8,2,10),(8,3,4),(8,4,6),(8,5,3),(8,6,8),(8,7,1),(8,8,2),(8,9,9),(8,10,7),
    (9,1,2),(9,2,8),(9,3,5),(9,4,10),(9,5,3),(9,6,7),(9,7,6),(9,8,4),(9,9,1),(9,10,9),
    (10,1,4),(10,2,6),(10,3,1),(10,4,9),(10,5,8),(10,6,3),(10,7,5),(10,8,2),(10,9,7),(10,10,10)
]
order_item_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("product_id", IntegerType(), False),
    StructField("qty", IntegerType(), False)
])
order_item_df = spark.createDataFrame(order_item_data, order_item_schema)

order_item_df.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_bronze.order_items")

#silver layer setup

In [0]:
orders_product = spark.sql("""
select 
A.*,
B.order_id, 
B.qty,
(B.qty * A.unit_price) as total_order_value,
C.customer_id,
C.order_date
from sales_catalog.sales_bronze.products A  
join sales_catalog.sales_bronze.order_items B 
on A.product_id = B.product_id 
join sales_catalog.sales_bronze.orders C 
on C.order_id = B.order_id 
""")

orders_product.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_silver.orders_product")

#gold layer setup

In [0]:
order_value_agg = spark.sql("""
select 
B.name as customer_name,
TO_CHAR(A.order_date, 'yyyy-MM') as year_month,
sum(A.total_order_value) as total_order_value_sum
from  sales_catalog.sales_silver.orders_product A 
join sales_catalog.sales_bronze.customers B
on A.customer_id = B.customer_id
group by year_month, B.customer_id, B.name
""")

order_value_agg.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_silver.order_value_agg")

In [0]:
higest_order_value_cust = spark.sql("""
with cte as (
select *,
row_number() over(partition by year_month order by total_order_value_sum desc) as ranking
from sales_catalog.sales_silver.order_value_agg
)

select year_month, customer_name, total_order_value_sum from cte where ranking = 1
""")


higest_order_value_cust.write.mode("overwrite").format("delta").saveAsTable("sales_catalog.sales_gold.higest_order_value_cust")

#lineage analysis

In [0]:
%sql
SELECT
  source_table_name,
  target_table_name
FROM system.access.table_lineage
WHERE target_table_full_name = 'sales_catalog.sales_gold.higest_order_value_cust'

In [0]:
%sql
SELECT
  source_table_name,
  source_column_name,
  target_table_name,
  target_column_name
FROM system.access.column_lineage
WHERE target_table_full_name = 'sales_catalog.sales_gold.higest_order_value_cust'
ORDER BY target_column_name;